# 쇼핑 데이터 전처리

1. 비짓제주 크롤링 데이터로 필터링
2. 탄소배출량 데이터 병합
3. 네이버맵 크롤링으로 데이터 보완
4. 주소 오류 보완

In [5]:
import pandas as pd
import numpy as np
import re
import os
import math
import json
import requests


import warnings
warnings.filterwarnings(action='ignore')

## 1. 비짓제주 크롤링 데이터로 필터링

In [6]:
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    return name.strip()

In [7]:
#주소 형식 맞추기
def addr_preprocessing(addr):
    addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시',addr)
    addr = re.sub('제주 제주시','제주특별자치도 제주시',addr)

    while (addr.split()[-1][-1].isdigit() == False) : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

In [8]:
vj_view = pd.read_csv('data/PRE_FINAL_JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv', encoding='utf-8-sig')
vj_craw_spot = pd.read_csv('data/VISITJEJU_쇼핑정보크롤링.csv')

In [9]:
# 원래 있던 기본 데이터에서 관광지만 남김
vj_view_spot = vj_view[vj_view['CL_NM']=='쇼핑']
vj_view_spot = vj_view_spot.reset_index(drop=True) 

#주소 전처리
vj_view_spot['ADDR_PRE'] = vj_view_spot['ADDR'].str.replace(pat="['",repl="",regex=False)
vj_view_spot['ADDR_PRE'] = vj_view_spot['ADDR_PRE'].str.replace(pat="']",repl="",regex=False)

vj_view_spot['ADDR_PRE'] = list(map(lambda x : addr_preprocessing(x), vj_view_spot['ADDR_PRE']))
vj_view_spot.head()

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE
0,b일상잡화점,쇼핑,['B일상잡화점'],['제주특별자치도 서귀포시 성산읍 오조로 93'],12129,18,30,49,48,43,60,제주특별자치도서귀포시성산읍오조로93
1,han컬렉션,쇼핑,['HAN 컬렉션'],['제주특별자치도 제주시 노연로 12'],12129,38,49,64,45,48,52,제주특별자치도제주시노연로12
2,갈중이,쇼핑,['갈중이'],['제주특별자치도 서귀포시 안덕면 사계남로216번길 24-61'],12129,24,43,60,46,34,40,제주특별자치도서귀포시안덕면사계남로216번길24-61
3,게른,쇼핑,['게른'],['제주특별자치도 서귀포시 중정로 1 '],12129,10,21,17,28,19,31,제주특별자치도서귀포시중정로1
4,곱은과일,쇼핑,['곱은과일'],['제주특별자치도 서귀포시 동문로 10'],12129,9,11,11,27,29,32,제주특별자치도서귀포시동문로10


In [10]:
vj_craw_spot = vj_craw_spot.drop(36, axis=0)

# 크롤링 데이터 이름 preprocessing -> 새로운 열 만듦
vj_craw_spot['name_PRE'] = list(map(lambda x : name_preprocessing(x), vj_craw_spot['name'])) ; vj_craw_spot

,name,stars,tags,address,telephone,like,heart,numreview,view,detail_text,rec_tour,rec_restaurant,rec_lodgment,rec_bigdata,detail_side_box,name_PRE
0,서귀포매일올레시장,5점,#쇼핑 #전통시장 #기념품 #상점/상가 #오일장 #관광기념품 #상점/상가 #오일장 ...,서귀포시 중앙로 62번길 18,(+82) 064-762-2925,33,715,113,"73,362",365일 문전성시를 이루는 서귀포시 대표 상설시장 ‘서귀포매일올레시장’은 서귀포시...,"['법화사', '신효 생활개선회', '정모시쉼터']","['세자매식당', '용이식당', '경성통닭']","['제주사랑 펜션', '비스타케이호텔월드컵', '굿데이 펜션']","['이중섭거리', '올레수산회센터', '달인수산']","{'소개': '365일 문전성시를 이루는 서귀포시 대표 상설시장', '상세 정보':...",서귀포매일올레시장
1,동문재래시장,4점,#쇼핑 #전통시장 #기념품 #밤 #상점/상가 #관광기념품 #밤 #상점/상가 #관광기념품,제주특별자치도 제주시 관덕로14길 20,(+82) 064-752-3001,55,"1,134",78,"103,629","가장 크고 역사가 깊은 재래시장 동문재래시장은 제주도 도심에 자리하였으며, 가장 ...","['서부두명품횟집거리', '천주교 순례길 - 김대건길', '제로스테이션(제로포인트트...","['명도암수다뜰', '미나리(제주시)', '라 아바나']","['스카이리더스호텔', '제주를여행하는히치하이커를위한안내소', '바다향기']","['오복떡집', '한성오메기떡', '코끼리통닭']","{'소개': '다양한 먹거리와 상업의 중심, 제주 동문 재래시장', '상세 정보':...",동문재래시장
2,제주관광공사 중문면세점,5점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,제주특별자치도 서귀포시 중문관광로 224(중문동) 제주국제컨벤션센터 1F,(+82) 064-780-7700,19,175,32,"65,515","제주 관광의 중심인 중문에 위치한, 국내 최초 내국인 지정면세점에서 여유롭게 쇼핑하...","['천제연폭포 & 제주4.3중문면희생자위령비', '제주 고유 식재료 ‘푸른 콩’으로...","['돈이랑 서귀포점', '중문대들보', '참이쁜돼지']","['제주하나호텔', '남쪽나라빌', '그랜드 조선 제주']","['델리지아레스토랑', '무비랜드왁스뮤지엄', '씨에스호텔앤리조트']",{'소개': '제주특별자치도 출자 지방공기업인 제주관광공사에서 운영하는 내국인 전용...,제주관광공사중문면세점
3,벨롱장,4점,#쇼핑 #플리마켓 #기념품 #관광기념품 #상점/상가,제주특별자치도 제주시 구좌읍 세화1길 20-8,(+82) --,9,69,22,"25,652","제주의 작은 바닷가 마을 세화리. 3월에서 10월까지 매주 토요일, 지역 주민과 여...","['세인트포골프장', '종달고망난돌쉼터', '진빌레 밭담길']","['너는파라다이스길리', '이찌마루', '세러데이아일랜드']","['디스이즈핫', '종달스토리', '제주올레하우스팬션']","['말이', '맛좋은디', '제주한잔, 세화']","{'소개': '세화리, 아름다운 바다 옆 벨롱장', '상세 정보': '시간과 장소는...",벨롱장
4,제주민속오일시장,5점,#쇼핑 #전통시장 #오일장 #상점/상가 #관광기념품,제주특별자치도 제주시 오일장서길 26,(+82) 064-743-5985,9,194,20,"60,556",제주 현지인의 삶을 가장 잘 말해주는 오일시장 전국에서 가장 규모가 큰 오일시장으...,"['관음사 군 주둔지 옛터', '문화공간 양', '서부두명품횟집거리']","['고로케야', '한량2010', '라 아바나']","['제주공항 고추잠자리 게스트하우스', '호텔 더블유 신제주점(hotel W)', ...","['승범이네집', '자연이야기', '길벗식당']","{'소개': '제주 현지인의 삶을 가장 잘 말해주는 오일시장', '상세 정보': '...",제주민속오일시장
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,한마음한라봉,NaN,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,제주특별자치도 제주시 애월읍 애월해안로 32,(+82) 064-799-3859,1,17,0,"6,769",제주영농조합법인에서 운영하는 한마음한라봉은 한라봉을 중심으로 다양한 품종의 귤을 판...,"['에버리스 골프리조트', '구엄포구', '쉼터마을 유수암리 농어촌체험마을']","['인디언키친', '보말고을', '귤따고 레드향갈고']","['시땅', '아마빌레', '잇츠힐']","['한마음한라봉', '문서프', '곽지해수욕장야영장']","{'소개': '제주영농조합법인에서 운영하는 제주특산물 판매처', '상세 정보': '...",한마음한라봉
130,한복담,NaN,#커플 #친구 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,제주특별자치도 제주시 신대로12길 51,(+82),0,5,0,"1,657",이곳은 접근이 쉬운 제주 시내에 위치해있다. 다양한 한복과 함께 액세서리도 준비되어...,"['문화공간 양', '주정공장(동척회사) 옛터', '서부두명품횟집거리']","['고로케야', '팝업 플라자', '촵촵']","['바다향기', '비앤비호텔', '스카이리더스호텔']","['빠빠라기', '제원분식', '한복담']","{'소개': '합리적인 가격으로 원하는 시간만큼 한복 체험을 해볼 수 있는 곳.',...",한복담
131,함덕민속오일시장,NaN,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,제주특별자치도 제주시 조천읍 함덕16길 15-13,(+82) 064-783-8559,0,6,0,"6,431",수수한 매력을 지니고 있는 전통시장 함덕오일시장은 약 40여 개의 점포에는 실생활...,"['목시물굴', '비자림농원', '북촌국민학교']","['아따블르', '호끌락다락', '금보가든']","['동백동산펜션', '오션그랜드호텔 (함덕)', '하루앤하루']","['준반점', '함덕카페거리', '함덕밀면']","{'소개': '수수한 매력을 지니고 있는 전통시장', '상세 정보': '매월 1일,...",함덕민속오일시장
132,현승이네농장,NaN,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,제주특별자치도 제주시 애월읍 신엄리 16,(+82) 010-9289-1285,0,5,0,"3,567","제주시 애월읍 해안 도로 인근에 위치한 현승이네 농장은 천혜향, 레드향, 황금향을 ...","['멍멍플레이스', '중엄리새물', '에버리스 골프리조트']","['산책하우스', '노리터스낵', '귤따고 레드향갈고']","['달자펜션', '씨스테이호텔', '폴라리스인제주']","['물메(물미, 수산봉)', '스테이달하', '물메랑']","{'소개': '애월 더럭분교 인근 위치한 귤농장.', '이용 시간': '평일 : 0...",현승이네농장


In [11]:
# 안 쓸 열들 drop
vj_craw_spot = vj_craw_spot.drop(['stars', 'address', 'telephone', 'numreview', 'view', 'rec_tour', 'rec_restaurant', 'rec_lodgment', 'rec_bigdata'], axis=1) ; vj_craw_spot

,name,tags,like,heart,detail_text,detail_side_box,name_PRE
0,서귀포매일올레시장,#쇼핑 #전통시장 #기념품 #상점/상가 #오일장 #관광기념품 #상점/상가 #오일장 ...,33,715,365일 문전성시를 이루는 서귀포시 대표 상설시장 ‘서귀포매일올레시장’은 서귀포시...,"{'소개': '365일 문전성시를 이루는 서귀포시 대표 상설시장', '상세 정보':...",서귀포매일올레시장
1,동문재래시장,#쇼핑 #전통시장 #기념품 #밤 #상점/상가 #관광기념품 #밤 #상점/상가 #관광기념품,55,"1,134","가장 크고 역사가 깊은 재래시장 동문재래시장은 제주도 도심에 자리하였으며, 가장 ...","{'소개': '다양한 먹거리와 상업의 중심, 제주 동문 재래시장', '상세 정보':...",동문재래시장
2,제주관광공사 중문면세점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,19,175,"제주 관광의 중심인 중문에 위치한, 국내 최초 내국인 지정면세점에서 여유롭게 쇼핑하...",{'소개': '제주특별자치도 출자 지방공기업인 제주관광공사에서 운영하는 내국인 전용...,제주관광공사중문면세점
3,벨롱장,#쇼핑 #플리마켓 #기념품 #관광기념품 #상점/상가,9,69,"제주의 작은 바닷가 마을 세화리. 3월에서 10월까지 매주 토요일, 지역 주민과 여...","{'소개': '세화리, 아름다운 바다 옆 벨롱장', '상세 정보': '시간과 장소는...",벨롱장
4,제주민속오일시장,#쇼핑 #전통시장 #오일장 #상점/상가 #관광기념품,9,194,제주 현지인의 삶을 가장 잘 말해주는 오일시장 전국에서 가장 규모가 큰 오일시장으...,"{'소개': '제주 현지인의 삶을 가장 잘 말해주는 오일시장', '상세 정보': '...",제주민속오일시장
...,...,...,...,...,...,...,...
129,한마음한라봉,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1,17,제주영농조합법인에서 운영하는 한마음한라봉은 한라봉을 중심으로 다양한 품종의 귤을 판...,"{'소개': '제주영농조합법인에서 운영하는 제주특산물 판매처', '상세 정보': '...",한마음한라봉
130,한복담,#커플 #친구 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0,5,이곳은 접근이 쉬운 제주 시내에 위치해있다. 다양한 한복과 함께 액세서리도 준비되어...,"{'소개': '합리적인 가격으로 원하는 시간만큼 한복 체험을 해볼 수 있는 곳.',...",한복담
131,함덕민속오일시장,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0,6,수수한 매력을 지니고 있는 전통시장 함덕오일시장은 약 40여 개의 점포에는 실생활...,"{'소개': '수수한 매력을 지니고 있는 전통시장', '상세 정보': '매월 1일,...",함덕민속오일시장
132,현승이네농장,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0,5,"제주시 애월읍 해안 도로 인근에 위치한 현승이네 농장은 천혜향, 레드향, 황금향을 ...","{'소개': '애월 더럭분교 인근 위치한 귤농장.', '이용 시간': '평일 : 0...",현승이네농장


In [12]:
#가능한 많은 데이터를 merge하기 위해 outer 활용
checker2 = pd.merge(left = vj_view_spot, right = vj_craw_spot, how = 'outer', left_on = 'AREA_NM_PRE', right_on = 'name_PRE').reset_index(drop=True)
checker2.tail(20)

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
124,한라꽃방,쇼핑,['한라꽃방'],['제주특별자치도 서귀포시 이중섭로 34'],12129.0,5.0,11.0,22.0,15.0,17.0,13.0,제주특별자치도서귀포시이중섭로34,한라꽃방,#꽃 #꽃집 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,1,대한민국 최남단에 위치해 있는 서귀포. 굽이굽이 놓인 길을 따라 한라산을 넘어 서귀...,{'소개': '과거 이중섭이 머물며 많은 작품을 남겼던 생가. 그 주변으로 조성된 ...,한라꽃방
125,한림매일시장,쇼핑,['한림매일시장'],['제주특별자치도 제주시 한림읍 한림로 687'],12129.0,66.0,71.0,195.0,60.0,82.0,86.0,제주특별자치도제주시한림읍한림로687,한림매일시장,#전통시장 #쇼핑 #상점/상가 #관광기념품 #관광기념품,2.0,5,옛 재래시장의 면모를 보여주는 한림매일시장 제주에서 가장 큰 읍인 한림읍 중심에는...,"{'소개': '옛 재래시장의 면모를 보여주는 한림매일시장', '상세 정보': '점포...",한림매일시장
126,한림민속오일시장,쇼핑,['한림민속오일시장'],['제주특별자치도 제주시 한림읍 한수풀로4길 10'],12129.0,144.0,175.0,361.0,130.0,152.0,153.0,제주특별자치도제주시한림읍한수풀로4길10,한림민속오일시장,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,4.0,19,여러분 곁에 가까이 있는 한림조끄뜨레시장 한림민속오일시장은 한림읍 주민들로 북적대...,"{'소개': '여러분 곁에 가까이 있는 한림조끄뜨레시장', '상세 정보': '장열리...",한림민속오일시장
127,한마음한라봉,쇼핑,['한마음한라봉'],['제주특별자치도 제주시 애월읍 애월해안로 32'],12129.0,18.0,27.0,24.0,30.0,45.0,52.0,제주특별자치도제주시애월읍애월해안로32,한마음한라봉,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1.0,17,제주영농조합법인에서 운영하는 한마음한라봉은 한라봉을 중심으로 다양한 품종의 귤을 판...,"{'소개': '제주영농조합법인에서 운영하는 제주특산물 판매처', '상세 정보': '...",한마음한라봉
128,한복담,쇼핑,['한복담'],['제주특별자치도 제주시 신대로12길 51'],12129.0,12.0,9.0,18.0,23.0,29.0,16.0,제주특별자치도제주시신대로12길51,한복담,#커플 #친구 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,5,이곳은 접근이 쉬운 제주 시내에 위치해있다. 다양한 한복과 함께 액세서리도 준비되어...,"{'소개': '합리적인 가격으로 원하는 시간만큼 한복 체험을 해볼 수 있는 곳.',...",한복담
129,함덕민속오일시장,쇼핑,['함덕민속오일시장'],['제주특별자치도 제주시 조천읍 함덕16길 15-13'],12129.0,91.0,119.0,204.0,66.0,79.0,72.0,제주특별자치도제주시조천읍함덕16길15-13,함덕민속오일시장,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0.0,6,수수한 매력을 지니고 있는 전통시장 함덕오일시장은 약 40여 개의 점포에는 실생활...,"{'소개': '수수한 매력을 지니고 있는 전통시장', '상세 정보': '매월 1일,...",함덕민속오일시장
130,헤라다플라워,쇼핑,['헤라다플라워'],['제주특별자치도 제주시 애월읍 도치돌길 284'],4042.0,0.0,0.0,0.0,0.0,0.0,0.0,제주특별자치도제주시애월읍도치돌길284,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,현승이네농장,쇼핑,['현승이네농장'],['제주특별자치도 제주시 애월읍 신엄리 16'],12129.0,7.0,28.0,19.0,29.0,25.0,26.0,제주특별자치도제주시애월읍신엄리16,현승이네농장,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0.0,5,"제주시 애월읍 해안 도로 인근에 위치한 현승이네 농장은 천혜향, 레드향, 황금향을 ...","{'소개': '애월 더럭분교 인근 위치한 귤농장.', '이용 시간': '평일 : 0...",현승이네농장
132,화북종합시장,쇼핑,['화북종합시장'],['제주특별자치도 제주시 진동로5길 31'],12129.0,26.0,44.0,12.0,0.0,21.0,76.0,제주특별자치도제주시진동로5길31,화북종합시장,#전통시장 #쇼핑 #상점/상가 #관광기념품 #관광기념품,0.0,0,정감 넘치는 동네시장 제주 화북동에 있는 동네 시장 ‘화북종합시장’은 1992년에...,"{'소개': '정감 넘치는 동네시장', '상세 정보': '운영시간 점포별 상이'}",화북종합시장
133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주관광공사 중문면세점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,19.0,175,"제주 관광의 중심인 중문에 위치한, 국내 최초 내국인 지정면세점에서 여유롭게 쇼핑하...",{'소개': '제주특별자치도 출자 지방공기업인 제주관광공사에서 운영하는 내국인 전용...,제주관광공사중문면세점


In [13]:
check_list=checker2[checker2.AREA_NM_PRE.isna()].name_PRE
name_list = []
idx=[]
for i in range(0,len(check_list)):
    ck = checker2[checker2.AREA_NM_PRE.isna()].name_PRE.iloc[i]
    check_df = checker2[(checker2['AREA_NM_PRE'].str.contains(ck,na=False)) & (checker2['name_PRE'].isna())]
    check_df = pd.concat([check_df,checker2[checker2['name_PRE'] == ck]], ignore_index=False)
   
    if len(check_df) == 2 :
        name_list.append(ck)
        idx.append(check_df.index)
print(len(name_list),len(idx))

1 1


In [14]:
checker2.iloc[idx[0]]

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
97,제주관광공사중문면세점내국인,쇼핑,['제주관광공사 중문면세점 (내국인)'],['제주특별자치도 서귀포시 중문관광로 224(중문동) 제주국제컨벤션센터 1F'],12129.0,513.0,595.0,829.0,773.0,994.0,888.0,제주특별자치도서귀포시중문관광로,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주관광공사 중문면세점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,19.0,175,"제주 관광의 중심인 중문에 위치한, 국내 최초 내국인 지정면세점에서 여유롭게 쇼핑하...",{'소개': '제주특별자치도 출자 지방공기업인 제주관광공사에서 운영하는 내국인 전용...,제주관광공사중문면세점


In [15]:
def adj_rows(main_idx, sub_idx):
    checker2.at[main_idx, 'name'] = checker2.at[sub_idx, 'name']
    checker2.at[main_idx, 'tags'] = checker2.at[sub_idx, 'tags']
    checker2.at[main_idx, 'like'] = checker2.at[sub_idx, 'like']
    checker2.at[main_idx, 'heart'] = checker2.at[sub_idx, 'heart']
    checker2.at[main_idx, 'name_PRE'] = checker2.at[sub_idx, 'name_PRE']

In [16]:
for k in range(0,len(idx)):
    adj_rows(idx[k][0], idx[k][1])

In [17]:
checker2.iloc[idx[0]] #수정된 내용 확인

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
97,제주관광공사중문면세점내국인,쇼핑,['제주관광공사 중문면세점 (내국인)'],['제주특별자치도 서귀포시 중문관광로 224(중문동) 제주국제컨벤션센터 1F'],12129.0,513.0,595.0,829.0,773.0,994.0,888.0,제주특별자치도서귀포시중문관광로,제주관광공사 중문면세점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,19.0,175,NaN,NaN,제주관광공사중문면세점
133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주관광공사 중문면세점,#쇼핑 #면세점 #기념품 #관광기념품 #상점/상가 #dutyfreeshop #관광기...,19.0,175,"제주 관광의 중심인 중문에 위치한, 국내 최초 내국인 지정면세점에서 여유롭게 쇼핑하...",{'소개': '제주특별자치도 출자 지방공기업인 제주관광공사에서 운영하는 내국인 전용...,제주관광공사중문면세점


In [18]:
checker2.columns

Index(['AREA_NM_PRE', 'CL_NM', 'AREA_NM', 'ADDR', 'BASE_YEAR', 'MAR_VIEW_CO',
       'APR_VIEW_CO', 'MAY_VIEW_CO', 'SEP_VIEW_CO', 'OCT_VIEW_CO',
       'NOV_VIEW_CO', 'ADDR_PRE', 'name', 'tags', 'like', 'heart',
       'detail_text', 'detail_side_box', 'name_PRE'],
      dtype='object')

In [19]:
checker2 = checker2.dropna(subset = ['AREA_NM_PRE', 'name_PRE']).reset_index(drop=True)
checker2 = checker2.drop(['CL_NM','AREA_NM','ADDR','BASE_YEAR','name'],axis = 1)
checker2

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,heart,detail_text,detail_side_box,name_PRE
0,b일상잡화점,18.0,30.0,49.0,48.0,43.0,60.0,제주특별자치도서귀포시성산읍오조로93,#쇼핑 #기념품 #관광기념품 #상점/상가,1.0,30,반복되는 일상에 작은 위트를 더해 특별한 일상을 만드는 곳 성산일출봉과 멀지 않은 ...,"{'소개': '일상 더하기 위트 = B일상 잡화점', '상세 정보': '일요일, 마...",b일상잡화점
1,han컬렉션,38.0,49.0,64.0,45.0,48.0,52.0,제주특별자치도제주시노연로12,#부모 #커플 #혼자 #친구 #쇼핑 #관광기념품 #상점/상가,1.0,7,200여 명의 대한민국 대표 K패션 디자이너들의 감각적이고 아방가르드한 작품들을 한...,{'소개': '국내외에서 가장 주목받고 있는 한국 패션 디자이너 200명의 작품을 ...,han컬렉션
2,갈중이,24.0,43.0,60.0,46.0,34.0,40.0,제주특별자치도서귀포시안덕면사계남로216번길24-61,#기념품 #쇼핑 #관광기념품 #상점/상가 #상점/상가,0.0,8,제주 갈옷을 대표하는 3대를 이어온 갈중이는 천연감물 염색부터 갈옷 디자인과 갈옷봉...,"{'소개': '제주전통 갈옷 전문 의류매장', '이용 시간': '평일 : 09:00...",갈중이
3,게른,10.0,21.0,17.0,28.0,19.0,31.0,제주특별자치도서귀포시중정로1,#소품샵 #고양이 #고양이소품 #고양이용품 #쇼핑 #관광기념품 #상점/상가,2.0,7,"게른은 '거리낌없이', '스스럼없이'란 뜻의 독일어로, 언제든 거림낌없이 오라는 주...","{'소개': '고양이에 대한 모든 것을 취급하는 고양이잡화점', '상세 정보': '...",게른
4,곱은과일,9.0,11.0,11.0,27.0,29.0,32.0,제주특별자치도서귀포시동문로10,#특산품 #기념품 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,6,곱은 과일은 서귀포 위미농협 감귤을 전문적으로 판매하고 있어 사계절 내내 제주 감귤...,"{'소개': '동문시장에 위치한 서귀포 위미농협 감귤 전문 판매장', '이용 시간'...",곱은과일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,한마음한라봉,18.0,27.0,24.0,30.0,45.0,52.0,제주특별자치도제주시애월읍애월해안로32,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1.0,17,제주영농조합법인에서 운영하는 한마음한라봉은 한라봉을 중심으로 다양한 품종의 귤을 판...,"{'소개': '제주영농조합법인에서 운영하는 제주특산물 판매처', '상세 정보': '...",한마음한라봉
119,한복담,12.0,9.0,18.0,23.0,29.0,16.0,제주특별자치도제주시신대로12길51,#커플 #친구 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,5,이곳은 접근이 쉬운 제주 시내에 위치해있다. 다양한 한복과 함께 액세서리도 준비되어...,"{'소개': '합리적인 가격으로 원하는 시간만큼 한복 체험을 해볼 수 있는 곳.',...",한복담
120,함덕민속오일시장,91.0,119.0,204.0,66.0,79.0,72.0,제주특별자치도제주시조천읍함덕16길15-13,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0.0,6,수수한 매력을 지니고 있는 전통시장 함덕오일시장은 약 40여 개의 점포에는 실생활...,"{'소개': '수수한 매력을 지니고 있는 전통시장', '상세 정보': '매월 1일,...",함덕민속오일시장
121,현승이네농장,7.0,28.0,19.0,29.0,25.0,26.0,제주특별자치도제주시애월읍신엄리16,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0.0,5,"제주시 애월읍 해안 도로 인근에 위치한 현승이네 농장은 천혜향, 레드향, 황금향을 ...","{'소개': '애월 더럭분교 인근 위치한 귤농장.', '이용 시간': '평일 : 0...",현승이네농장


In [21]:
#checker2.to_csv('PRE_1_쇼핑데이터1011.csv', encoding='utf-8-sig', index=False)

In [22]:
place_df = checker2

In [23]:
# 전처리 오류로 주소가 정보가 충분하지 않은 경우 카카오API결과를 충분히 신뢰할 수 없으므로, POI데이터를 활용하여 보완한다.
no_number_index = place_df[-place_df['ADDR_PRE'].str[-1].str.isdigit()].index
place_df[['AREA_NM_PRE','ADDR_PRE']].iloc[no_number_index]

,AREA_NM_PRE,ADDR_PRE
41,보미다,제주특별자치도서귀포시이어도로
89,제주관광공사중문면세점내국인,제주특별자치도서귀포시중문관광로
101,차귀도오징어,제주특별자치도제주시한경면


In [24]:
# 전처리 오류로 한글에서 끊긴 경우 직접 데이터로 보완한다.
place_df['ADDR_PRE'].iloc[no_number_index[0]] = "제주특별자치도서귀포시이어도로936-1"
place_df['ADDR_PRE'].iloc[no_number_index[1]] = "제주특별자치도서귀포시중문관광로224"
place_df = place_df.drop(101, axis = 0).reset_index(drop=True)

In [25]:
#카카오api로 지번주소, 위도, 경도를 가져오겠다
api_key = '7465f66e54fbe4f71543e474623685f1' #숨김처리

In [26]:
place_df['LTNO_ADDR'] = np.NaN
place_df['GPS_LNGTD'] = np.NaN
place_df['GPS_LTTD'] = np.NaN

addrs = place_df.ADDR_PRE

for i in range(0,len(addrs)) :
    try : 
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addrs[i])
        headers = {"Authorization": "KakaoAK " + api_key}
        result = json.loads(str(requests.get(url, headers = headers).text))
    
        place_df['LTNO_ADDR'].iloc[i] = result['documents'][0]['address']['address_name']
        place_df['GPS_LNGTD'].iloc[i] = result['documents'][0]['address']['x'] #경도
        place_df['GPS_LTTD'].iloc[i] = result['documents'][0]['address']['y'] #위도
    
    except: #카카오맵에서 검색 안됨
        place_df['LTNO_ADDR'].iloc[i] = np.NaN
        place_df['GPS_LNGTD'].iloc[i] = np.NaN
        place_df['GPS_LTTD'].iloc[i] = np.NaN

In [30]:
idx = place_df[place_df.GPS_LNGTD.isna()].index

In [31]:
#직접 넣었다
place_df['GPS_LNGTD'][idx[0]] = 126.2431583 #서쪽가게
place_df['GPS_LTTD'][idx[0]] = 33.3957313
place_df['LTNO_ADDR'][idx[0]] = "제주특별자치도제주시협재리1857-1"

place_df['GPS_LNGTD'][idx[1]] = 126.4785578 #자연이야기
place_df['GPS_LTTD'][idx[1]] = 33.4894641 
place_df['ADDR_PRE'][idx[1]] = "제주특별자치도제주시월랑로6길21"
place_df['LTNO_ADDR'][idx[1]] = "제주특별자치도제주시노형동1052-27"


place_df['GPS_LNGTD'][idx[2]] = 126.7953956 #팰롱팰롱빛나는
place_df['GPS_LTTD'][idx[2]] = 33.5557
place_df['LTNO_ADDR'][idx[2]] = "제주특별자치도제주시구좌읍월정리546"

In [33]:
place_df = place_df.drop(19, axis = 0)

In [32]:
#place_df.to_csv('PRE_3_쇼핑데이터1011.csv', encoding='utf-8-sig', index=False)

## 2. 탄소배출량 데이터 병합

In [38]:
#place_df = pd.read_csv('PRE_3_쇼핑데이터1011.csv',encoding='utf-8-sig')
emit = pd.read_csv('data/PRE_CO2_지번단위탄소배출량_meanmax.csv',encoding='utf-8-sig')
emit = emit[['LTNO_ADDR', 'co2_max']]

### 1) 탄소배출 데이터 전처리

In [39]:
#주소 형식 맞추기
def addr_preprocessing3(addr):

    while addr.split()[-1][-1].isdigit() == False : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

In [40]:
#주소 정리
place_df['LTNO_ADDR'] = list(map(lambda x : addr_preprocessing3(x), place_df['LTNO_ADDR']))

emit['LTNO_ADDR']=emit['LTNO_ADDR'].str.replace(pat='번지',repl='',regex=False)
emit['LTNO_ADDR'] = list(map(lambda x : addr_preprocessing3(x), emit['LTNO_ADDR'])) ; emit

,LTNO_ADDR,co2_max
0,제주특별자치도서귀포시강정동0,0.009123
1,제주특별자치도서귀포시강정동1026-1,0.109615
2,제주특별자치도서귀포시강정동1045-5,0.000990
3,제주특별자치도서귀포시강정동108-10,0.054986
4,제주특별자치도서귀포시강정동1155,0.002114
...,...,...
34515,제주특별자치도제주시회천동656-2,0.080382
34516,제주특별자치도제주시회천동722-2,0.005129
34517,제주특별자치도제주시회천동742-2,0.002098
34518,제주특별자치도제주시회천동79-2,0.031270


In [41]:
all_df = pd.merge(left = place_df, right = emit, how = 'left', on = 'LTNO_ADDR').reset_index(drop=True)
all_df

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,heart,detail_text,detail_side_box,name_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_max
0,b일상잡화점,18.0,30.0,49.0,48.0,43.0,60.0,제주특별자치도서귀포시성산읍오조로93,#쇼핑 #기념품 #관광기념품 #상점/상가,1.0,30,반복되는 일상에 작은 위트를 더해 특별한 일상을 만드는 곳 성산일출봉과 멀지 않은 ...,"{'소개': '일상 더하기 위트 = B일상 잡화점', '상세 정보': '일요일, 마...",b일상잡화점,제주특별자치도서귀포시성산읍오조리747-4,126.914635364691,33.4620476392636,0.002689
1,han컬렉션,38.0,49.0,64.0,45.0,48.0,52.0,제주특별자치도제주시노연로12,#부모 #커플 #혼자 #친구 #쇼핑 #관광기념품 #상점/상가,1.0,7,200여 명의 대한민국 대표 K패션 디자이너들의 감각적이고 아방가르드한 작품들을 한...,{'소개': '국내외에서 가장 주목받고 있는 한국 패션 디자이너 200명의 작품을 ...,han컬렉션,제주특별자치도제주시노형동925,126.481459449471,33.4852623358443,NaN
2,갈중이,24.0,43.0,60.0,46.0,34.0,40.0,제주특별자치도서귀포시안덕면사계남로216번길24-61,#기념품 #쇼핑 #관광기념품 #상점/상가 #상점/상가,0.0,8,제주 갈옷을 대표하는 3대를 이어온 갈중이는 천연감물 염색부터 갈옷 디자인과 갈옷봉...,"{'소개': '제주전통 갈옷 전문 의류매장', '이용 시간': '평일 : 09:00...",갈중이,제주특별자치도서귀포시안덕면사계리113,126.313645335479,33.2351178178847,0.066441
3,게른,10.0,21.0,17.0,28.0,19.0,31.0,제주특별자치도서귀포시중정로1,#소품샵 #고양이 #고양이소품 #고양이용품 #쇼핑 #관광기념품 #상점/상가,2.0,7,"게른은 '거리낌없이', '스스럼없이'란 뜻의 독일어로, 언제든 거림낌없이 오라는 주...","{'소개': '고양이에 대한 모든 것을 취급하는 고양이잡화점', '상세 정보': '...",게른,제주특별자치도서귀포시서귀동323-19,126.556567312748,33.2477083075349,0.148283
4,곱은과일,9.0,11.0,11.0,27.0,29.0,32.0,제주특별자치도서귀포시동문로10,#특산품 #기념품 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,6,곱은 과일은 서귀포 위미농협 감귤을 전문적으로 판매하고 있어 사계절 내내 제주 감귤...,"{'소개': '동문시장에 위치한 서귀포 위미농협 감귤 전문 판매장', '이용 시간'...",곱은과일,제주특별자치도서귀포시서귀동286-5,126.562244724595,33.2519028780252,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,한마음한라봉,18.0,27.0,24.0,30.0,45.0,52.0,제주특별자치도제주시애월읍애월해안로32,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1.0,17,제주영농조합법인에서 운영하는 한마음한라봉은 한라봉을 중심으로 다양한 품종의 귤을 판...,"{'소개': '제주영농조합법인에서 운영하는 제주특산물 판매처', '상세 정보': '...",한마음한라봉,제주특별자치도제주시애월읍애월리1827-1,126.318029771951,33.4664978912623,NaN
117,한복담,12.0,9.0,18.0,23.0,29.0,16.0,제주특별자치도제주시신대로12길51,#커플 #친구 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,5,이곳은 접근이 쉬운 제주 시내에 위치해있다. 다양한 한복과 함께 액세서리도 준비되어...,"{'소개': '합리적인 가격으로 원하는 시간만큼 한복 체험을 해볼 수 있는 곳.',...",한복담,제주특별자치도제주시연동272-33,126.491113245438,33.4885711655246,1.614905
118,함덕민속오일시장,91.0,119.0,204.0,66.0,79.0,72.0,제주특별자치도제주시조천읍함덕16길15-13,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0.0,6,수수한 매력을 지니고 있는 전통시장 함덕오일시장은 약 40여 개의 점포에는 실생활...,"{'소개': '수수한 매력을 지니고 있는 전통시장', '상세 정보': '매월 1일,...",함덕민속오일시장,제주특별자치도제주시조천읍함덕리972-7,126.667960660289,33.5402526599241,0.024546
119,현승이네농장,7.0,28.0,19.0,29.0,25.0,26.0,제주특별자치도제주시애월읍신엄리16,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0.0,5,"제주시 애월읍 해안 도로 인근에 위치한 현승이네 농장은 천혜향, 레드향, 황금향을 ...","{'소개': '애월 더럭분교 인근 위치한 귤농장.', '이용 시간': '평일 : 0...",현승이네농장,제주특별자치도제주시애월읍신엄리16,126.378786011688,33.4636139040508,NaN


In [42]:
all_df.co2_max.isna().sum() 

54

### 2) 데이터 정리

In [43]:
#조회수는 총합으로 정리
all_df['VJ_VIEW'] = all_df[['MAR_VIEW_CO','APR_VIEW_CO', 'MAY_VIEW_CO', 'SEP_VIEW_CO', 'OCT_VIEW_CO','NOV_VIEW_CO']].sum(axis=1)
all_df.to_csv('data/쇼핑데이터_co2.csv', encoding='utf-8-sig', index=False) 

## 3. 네이버맵 크롤링으로  데이터 보완

In [46]:
#all_df = pd.read_csv('PRE_5쇼핑데이터_1011.csv', encoding='utf-8-sig')

naver_craw = pd.read_csv('data/쇼핑데이터_네이버크롤링.csv', encoding='utf-8-sig')

naver_craw = naver_craw.drop_duplicates(['org_list','name_list'])
naver_craw['name_list'] = list(map(lambda x : name_preprocessing(x), naver_craw['name_list']))

naver_craw['address_list'] = naver_craw['address_list'].str.replace(pat="제주 ",repl="제주특별자치도 ",regex=False)
naver_craw['address_list'] = naver_craw['address_list'].str.replace(pat="제주특별자치 ",repl="제주특별자치도 ",regex=False)

In [47]:
place = pd.merge(left = all_df, right = naver_craw, how="left", left_on=["AREA_NM_PRE"], right_on=["org_list"])
place = place.dropna(subset = ['name_list', 'address_list']).reset_index(drop=True) #검색 안된애는 삭제

In [48]:
# 크롤링 데이터의 적절성 확인1 : 주소가 일치하는가
addr_pre =place.address_list.str[0:15].str.replace(" ","")
chk_index1 = place[(place.ADDR_PRE.str[0:13] != addr_pre) * (place.LTNO_ADDR.str[0:13] != addr_pre)].index

# 크롤링 데이터의 적절성 확인2 : 이름이 일치하는가
chk_index2 = []
for i in range(0,len(place)):
    chk_name = place.name_list.iloc[i]
    
    if (place.AREA_NM_PRE.iloc[i] not in chk_name) * (place.name_PRE.iloc[i] not in chk_name)  * (chk_name not in place.name_PRE.iloc[i])  != 0 : #어디도 포함되지 않으면
        chk_index2.append(i)    
        
chk_index_inter = list(set(chk_index1) & set(chk_index2))
place[['name_PRE','name_list','LTNO_ADDR','address_list']].iloc[chk_index_inter]

,name_PRE,name_list,LTNO_ADDR,address_list
36,뷰티풀제주,뷰티풀리조트,제주특별자치도제주시용담삼동519-2,제주특별자치도 서귀포시 이어도로253번길 2
5,구름두리,용연구름다리,제주특별자치도서귀포시서귀동276-19,제주특별자치도 제주시 용담이동 2581
72,자연이야기,자연샵이야기,제주특별자치도제주시노형동1052-27,제주특별자치도 제주시 사장1길 31
10,당근가게,하우스레서피,제주특별자치도제주시용담일동2828-1,제주특별자치도 제주시 한림읍 일주서로 5892
14,동문시장,동문재래시장,제주특별자치도제주시일도일동1148-2,제주특별자치도 제주시 관덕로14길 20
78,제주민속오일시장,세화민속오일시장,제주특별자치도제주시도두일동1212,제주특별자치도 제주시 구좌읍 세화리 1500-44
88,카카오프렌즈스토어제주,카카오프렌즈제주프리미엄전문점,제주특별자치도제주시영평동2181,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1층 301호
26,미앤제주,미엔미치과의원,제주특별자치도제주시한림읍금악리산30-12,제주특별자치도 제주시 우정로15길 9


In [49]:
del_list = ["뷰티풀제주", "구름두리","자연이야기", "당근가게","제주민속오일시장","카카오프렌즈스토어제주","미앤제주"]

del_index = []
for i in chk_index_inter :
    if place.name_PRE.iloc[i] in del_list :
        del_index.append(i)

place = place.drop(del_index, axis=0).reset_index(drop=True)

re_idx = place[place['name_PRE']=="동문수산시장"].index
place['LTNO_ADDR'].iloc[re_idx] = place['address_list'].iloc[re_idx] 
place['GPS_LNGTD'].iloc[re_idx]  = 126.5264799
place['GPS_LTTD'].iloc[re_idx] = 33.5120485

place

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,...,type_list,address_list,visitor_review_num_list,blog_review_num_list,visitor_review_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list
0,han컬렉션,38.0,49.0,64.0,45.0,48.0,52.0,제주특별자치도제주시노연로12,#부모 #커플 #혼자 #친구 #쇼핑 #관광기념품 #상점/상가,1.0,...,종합패션,제주특별자치도 제주시 노연로 12,334.0,10.0,no_result,NaN,NaN,NaN,NaN,han컬렉션
1,갈중이,24.0,43.0,60.0,46.0,34.0,40.0,제주특별자치도서귀포시안덕면사계남로216번길24-61,#기념품 #쇼핑 #관광기념품 #상점/상가 #상점/상가,0.0,...,의류제작,제주특별자치도 서귀포시 안덕면 사계남로216번길 24-61,49.0,31.0,no_result,NaN,NaN,NaN,NaN,갈중이
2,게른,10.0,21.0,17.0,28.0,19.0,31.0,제주특별자치도서귀포시중정로1,#소품샵 #고양이 #고양이소품 #고양이용품 #쇼핑 #관광기념품 #상점/상가,2.0,...,인테리어소품,제주특별자치도 서귀포시 효돈로 34 게른,21.0,61.0,no_result,NaN,NaN,NaN,NaN,게른
3,곱은과일,9.0,11.0,11.0,27.0,29.0,32.0,제주특별자치도서귀포시동문로10,#특산품 #기념품 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,...,과일,제주특별자치도 제주시 동문로 10,417.0,5.0,"{'과일이 맛있어요': 65, '신선해요': 48, '친절해요': 39, '제철 과...",NaN,NaN,NaN,NaN,곱은과일
4,곱은달사진관,29.0,30.0,42.0,40.0,48.0,63.0,제주특별자치도제주시조천읍곱은달길24,#흑백사진관 #스튜디오 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,...,"사진,스튜디오",제주특별자치도 제주시 조천읍 곱은달길 24,24.0,44.0,no_result,NaN,NaN,NaN,NaN,곱은달사진관
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,한림민속오일시장,144.0,175.0,361.0,130.0,152.0,153.0,제주특별자치도제주시한림읍한수풀로4길10,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,4.0,...,시장,제주특별자치도 제주시 한림읍 한수풀로4길 10,12.0,336.0,no_result,NaN,NaN,NaN,NaN,한림민속오일시장
92,한마음한라봉,18.0,27.0,24.0,30.0,45.0,52.0,제주특별자치도제주시애월읍애월해안로32,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1.0,...,과일,제주특별자치도 제주시 애월읍 애월해안로 32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,한마음한라봉
93,함덕민속오일시장,91.0,119.0,204.0,66.0,79.0,72.0,제주특별자치도제주시조천읍함덕16길15-13,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0.0,...,시장,제주특별자치도 제주시 조천읍 함덕16길 15-13,0.0,54.0,NaN,NaN,NaN,NaN,NaN,함덕민속오일시장
94,현승이네농장,7.0,28.0,19.0,29.0,25.0,26.0,제주특별자치도제주시애월읍신엄리16,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0.0,...,과일,제주특별자치도 제주시 애월읍 신엄리 16,0.0,1.0,NaN,NaN,NaN,NaN,NaN,현승이네농장


In [50]:
place['CAT_PRE'] = np.NaN

#직접 분류
for i in range(0,len(place)) :
    
    if place.type_list[i] in ["시장","종합분식"] :
        place['CAT_PRE'].iloc[i] = "시장"
        
    elif place.type_list[i] in ["인테리어소품", "판촉,기념품", "문구,팬시용품", "아워키친"] :
        place['CAT_PRE'].iloc[i] = "기념품/소품"

    elif place.type_list[i] in ["특산물,관광민예품", "과일", "유기농산물", " 떡,한과","슈퍼,마트","가공식품","수산물","초콜릿전문점","식료품제조","농수산물"] :
        place['CAT_PRE'].iloc[i] = "특산물"

    elif place.type_list[i] in ["서점", "독립서점"] :
        place['CAT_PRE'].iloc[i] = "서점"
        
    elif place.type_list[i] in ["공방","목공예품","금속공예"] :
        place['CAT_PRE'].iloc[i] = "공방"

    elif place.type_list[i] in ["패션잡화","아울렛","패션","의류제작","구제의류","종합패션","쇼핑센터,할인매장","상가,아케이드"] :
         place['CAT_PRE'].iloc[i] = "의류잡화"

In [51]:
place = place[-place['CAT_PRE'].isna()].reset_index(drop=True)
place.shape

(77, 31)

In [52]:
co2_mean = place[['CAT_PRE','co2_max']].groupby(['CAT_PRE']).mean()
co2_mean = co2_mean.reset_index(drop=False)
co2_mean

,CAT_PRE,co2_max
0,공방,0.016121
1,기념품/소품,0.118939
2,서점,0.052716
3,시장,0.345273
4,의류잡화,1.022089
5,특산물,0.878523


In [53]:
co2_na = place[place.co2_max.isna()].index

for i in co2_na:
    value = co2_mean[co2_mean.CAT_PRE == place.CAT_PRE.iloc[i]].co2_max.values[0]
    place['co2_max'].iloc[i] = value

In [54]:
place.isna().sum()

AREA_NM_PRE                 0
MAR_VIEW_CO                 0
APR_VIEW_CO                 0
MAY_VIEW_CO                 0
SEP_VIEW_CO                 0
OCT_VIEW_CO                 0
NOV_VIEW_CO                 0
ADDR_PRE                    0
tags                        0
like                        0
heart                       0
detail_text                 0
detail_side_box             0
name_PRE                    0
LTNO_ADDR                   0
GPS_LNGTD                   0
GPS_LTTD                    0
co2_max                     0
VJ_VIEW                     0
name_list                   0
type_list                   0
address_list                0
visitor_review_num_list     0
blog_review_num_list        0
visitor_review_list        25
dl_mood_list               77
dl_topic_list              77
dl_why_list                77
dl_20_pop_list             77
org_list                    0
CAT_PRE                     0
dtype: int64

In [55]:
#place.to_csv('PRE_쇼핑데이터_1011.csv', encoding='utf-8-sig', index=False) 
place_df = place

## 4.  주소 오류 보완

In [58]:
#place_df = pd.read_csv('PRE_쇼핑데이터_1011.csv',encoding='utf-8-sig')
vj_craw_spot = pd.read_csv('data/VISITJEJU_쇼핑정보크롤링.csv')

In [59]:
vj_craw_add = vj_craw_spot[['name','address']]
vj_craw_add

,name,address
0,서귀포매일올레시장,서귀포시 중앙로 62번길 18
1,동문재래시장,제주특별자치도 제주시 관덕로14길 20
2,제주관광공사 중문면세점,제주특별자치도 서귀포시 중문관광로 224(중문동) 제주국제컨벤션센터 1F
3,벨롱장,제주특별자치도 제주시 구좌읍 세화1길 20-8
4,제주민속오일시장,제주특별자치도 제주시 오일장서길 26
...,...,...
129,한마음한라봉,제주특별자치도 제주시 애월읍 애월해안로 32
130,한복담,제주특별자치도 제주시 신대로12길 51
131,함덕민속오일시장,제주특별자치도 제주시 조천읍 함덕16길 15-13
132,현승이네농장,제주특별자치도 제주시 애월읍 신엄리 16


In [60]:
vj_craw_add = vj_craw_spot[['name','address']]
vj_craw_add = vj_craw_add.drop(36, axis=0)
vj_craw_add['name'] = list(map(lambda x : name_preprocessing(x), vj_craw_add['name']))
vj_craw_add.columns = ['name_PRE','address']
vj_craw_add = vj_craw_add.drop_duplicates()

In [61]:
place_df2 = pd.merge(left = place_df, right = vj_craw_add, how = "left", on = "name_PRE")    
place_df2

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,...,visitor_review_num_list,blog_review_num_list,visitor_review_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list,CAT_PRE,address
0,han컬렉션,38.0,49.0,64.0,45.0,48.0,52.0,제주특별자치도제주시노연로12,#부모 #커플 #혼자 #친구 #쇼핑 #관광기념품 #상점/상가,1.0,...,334.0,10.0,no_result,NaN,NaN,NaN,NaN,han컬렉션,의류잡화,제주특별자치도 제주시 노연로 12
1,갈중이,24.0,43.0,60.0,46.0,34.0,40.0,제주특별자치도서귀포시안덕면사계남로216번길24-61,#기념품 #쇼핑 #관광기념품 #상점/상가 #상점/상가,0.0,...,49.0,31.0,no_result,NaN,NaN,NaN,NaN,갈중이,의류잡화,제주특별자치도 서귀포시 안덕면 사계남로216번길 24-61
2,게른,10.0,21.0,17.0,28.0,19.0,31.0,제주특별자치도서귀포시중정로1,#소품샵 #고양이 #고양이소품 #고양이용품 #쇼핑 #관광기념품 #상점/상가,2.0,...,21.0,61.0,no_result,NaN,NaN,NaN,NaN,게른,기념품/소품,제주특별자치도 서귀포시 중정로 1
3,곱은과일,9.0,11.0,11.0,27.0,29.0,32.0,제주특별자치도서귀포시동문로10,#특산품 #기념품 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,...,417.0,5.0,"{'과일이 맛있어요': 65, '신선해요': 48, '친절해요': 39, '제철 과...",NaN,NaN,NaN,NaN,곱은과일,특산물,제주특별자치도 서귀포시 동문로 10
4,기영상회,11.0,19.0,23.0,23.0,18.0,22.0,제주특별자치도제주시한림읍한림로345,#쇼핑 #슈퍼마켓 #나들가게 #관광기념품 #상점/상가 #관광기념품 #상점/상가,0.0,...,115.0,72.0,"{'상품이 다양해요': 18, '친절해요': 3, '신상품이 많아요': 3, '고기...",NaN,NaN,NaN,NaN,기영상회,특산물,제주특별자치도 제주시 한림읍 한림로 345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,한림민속오일시장,144.0,175.0,361.0,130.0,152.0,153.0,제주특별자치도제주시한림읍한수풀로4길10,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,4.0,...,12.0,336.0,no_result,NaN,NaN,NaN,NaN,한림민속오일시장,시장,제주특별자치도 제주시 한림읍 한수풀로4길 10
73,한마음한라봉,18.0,27.0,24.0,30.0,45.0,52.0,제주특별자치도제주시애월읍애월해안로32,#특산품 #한라봉 #쇼핑 #관광기념품 #상점/상가 #관광기념품 #상점/상가,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,한마음한라봉,특산물,제주특별자치도 제주시 애월읍 애월해안로 32
74,함덕민속오일시장,91.0,119.0,204.0,66.0,79.0,72.0,제주특별자치도제주시조천읍함덕16길15-13,#전통시장 #쇼핑 #상점/상가 #오일장 #관광기념품 #오일장 #관광기념품,0.0,...,0.0,54.0,NaN,NaN,NaN,NaN,NaN,함덕민속오일시장,시장,제주특별자치도 제주시 조천읍 함덕16길 15-13
75,현승이네농장,7.0,28.0,19.0,29.0,25.0,26.0,제주특별자치도제주시애월읍신엄리16,#천혜향 #한라봉 #레드향 #특산품 #쇼핑 #관광기념품 #상점/상가 #특산품 #쇼핑...,0.0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,현승이네농장,특산물,제주특별자치도 제주시 애월읍 신엄리 16


In [62]:
na_index = place_df2[place_df2.address.isna()].index
place_df2['address'].iloc[na_index] = place_df2['LTNO_ADDR'].iloc[na_index]

In [64]:
place_df2.to_csv('data/주소보완_쇼핑데이터.csv', encoding='utf-8-sig', index=False)